In [47]:
import torch
import torch.nn as nn
# 1. 单向.单层RNN
single_rnn = nn.RNN(4, 3, 1, batch_first=True)
input = torch.randn(1, 2, 4)# bs*sl*fs
out, h_n = single_rnn(input)
print(out)
print(h_n)

tensor([[[ 0.3982,  0.4844,  0.8871],
         [ 0.2332, -0.9083,  0.9175]]], grad_fn=<TransposeBackward1>)
tensor([[[ 0.2332, -0.9083,  0.9175]]], grad_fn=<StackBackward0>)


In [48]:
# 2. 双向.单层RNN
bidirectional_rnn = nn.RNN(4, 3, 1, batch_first=True, bidirectional=True)
out, h_n = bidirectional_rnn(input)
print(out.shape)
print(h_n.shape)

torch.Size([1, 2, 6])
torch.Size([2, 1, 3])


> 手动实现

In [49]:
bs, T = 2, 3 # 批大小，输入序列长度
input_size, hidden_size = 2, 3 # 输入特征大小， 隐藏层特征大小
input = torch.randn(bs, T, input_size) # 随机生成一个输入特征序列
h_prev = torch.zeros(bs, hidden_size) # 生成初始隐含状态
single_rnn = nn.RNN(input_size, hidden_size, 1, batch_first=True)
out, h_n = single_rnn(input, h_prev.unsqueeze(0))
print(input.shape)
print(out.shape)
print(h_n.shape)
def rnn_forward(input, weight_ih, weight_hh, bais_ih, bias_hh, h_prev):
    bs, T, input_size = input.shape
    h_dim = weight_ih.shape[0]
    
    pass

torch.Size([2, 3, 2])
torch.Size([2, 3, 3])
torch.Size([1, 2, 3])
